In [602]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [607]:
from planner import JourneyPlanner
from collections import defaultdict
import pickle

# Loading timetable

In [718]:
with open('../data/footpath_map.pickle', 'rb') as f:
    footpaths = pickle.load(f)

In [719]:
with open('../data/trips.pickle', 'rb') as f:
    trips = pickle.load(f)

In [720]:
with open('../data/stops.pickle', 'rb') as f:
    stops = pickle.load(f)

In [746]:
with open('../data/connections_data.pickle', 'rb') as f:
    connections = pickle.load(f)

In [722]:
paper_conn = [[
    ("y", "t", 100000, 110000, "1"),
    ("z", "t", 90000, 120000, "2"),
    ("x", "t", 80000, 130000, "3"),
    ("x", "y", 80000, 90000, "4"),
    ("s", "z", 70000, 80000, "5"),
    ("s", "x", 60000, 70000, "6"),
    ("s", "t", 50000, 140000, "7")
]]
timetable = (None, paper_conn, None, defaultdict(lambda: list()))

In [748]:
timetable = stops, connections, trips, footpaths

# Plan a route

In [749]:
planner = JourneyPlanner(timetable)

planner.plan_route(0, "8014558", "8591384", 1653141600, 1653163540)

In [710]:
planner.plan_route(0, "s", "t", 40000, 150000)

In [750]:
connections

[[('8575927', '8594339', 1653163140, 1653163140, '221.TA.26-640-j19-1.4.R'),
  ('8591147', '8591162', 1653163140, 1653163140, '120.TA.26-701-j19-1.5.R'),
  ('8576498', '8572601', 1653163140, 1653163140, '21.TA.1-338-j19-1.1.H'),
  ('8587652', '8591194', 1653163140, 1653163140, '2507.TA.26-9-B-j19-1.41.R'),
  ('8591175', '8591273', 1653163140, 1653163140, '165.TA.26-61-j19-1.2.R'),
  ('8591374', '8591328', 1653163140, 1653163140, '205.TA.26-4-j19-1.17.H'),
  ('8576198', '8591094', 1653163140, 1653163140, '773.TA.26-33E-j19-1.8.H'),
  ('8503709', '8574684', 1653163140, 1653163140, '583.TA.26-201-j19-1.2.H'),
  ('8503709', '8574684', 1653163140, 1653163140, '667.TA.26-201-j19-1.2.H'),
  ('8503709', '8574684', 1653163140, 1653163140, '577.TA.26-201-j19-1.2.H'),
  ('8503709', '8574684', 1653163140, 1653163140, '673.TA.26-201-j19-1.2.H'),
  ('8591065:0:D', '8590566', 1653163140, 1653163140, '73.TA.26-751-j19-1.2.H'),
  ('8591175', '8591319', 1653163140, 1653163140, '398.TA.26-61-j19-1.3.H'),